## Resumen del problema

Seleccionar un subconjunto de variables que maximice el rendimiento de un clasificador (por ejemplo, un árbol de decisión), usando un dataset real y evaluando la precisión de clasificación como medida de aptitud (fitness).

Algoritmos metaheurísticos :

1. Recocido simulado (Simulated Annealing - SA)
2. Algoritmo genético (Genetic Algorithm - GA)

### Dataset base para pruebas

Usaremos el dataset breast_cancer de sklearn.datasets (es pequeño, multivariable y típico para evaluación de selección de características).

### Configuración general antes de los algoritmos

### Importar librerías

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
# Carga un conjunto de datos real de cáncer de mama incluido en scikit-learn.
# Este dataset contiene 30 características (variables) que se usarán como base
# para aplicar selección de variables con metaheurísticas.

from sklearn.model_selection import cross_val_score
# Permite evaluar el rendimiento de un modelo con validación cruzada.
# Se usa para medir qué tan buena es una solución (subconjunto de variables),
# calculando la precisión promedio en varias particiones de los datos.

from sklearn.tree import DecisionTreeClassifier
# Clasificador basado en árboles de decisión.
# En este caso, lo usamos como modelo base para evaluar la calidad
# de los subconjuntos de variables seleccionados por los algoritmos.

### Cargar datos reales

In [ ]:
data = load_breast_cancer()
# Carga el dataset de cáncer de mama desde sklearn.
# Este conjunto de datos incluye características de imágenes digitales de células
# y una etiqueta binaria (maligno o benigno) como variable objetivo.

X = data.data
# Extrae las variables predictoras (matriz de características).
# Cada fila representa un paciente y cada columna una característica (por ejemplo, textura, área, simetría).

y = data.target
# Extrae la variable objetivo (etiquetas: 0 = maligno, 1 = benigno).

n_features = X.shape[1]
# Calcula el número total de características (columnas) disponibles en X.
# Esto es útil para saber cuántas variables se pueden seleccionar durante la optimización.

### Convertir a DataFrame para visualizar el conjunto de datos

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)

# Agregar la columna de etiquetas (diagnóstico: maligno o benigno)
df['target'] = data.target

# Mostrar las primeras 5 filas
df.head()

### Dimensiones del dataset

In [ ]:
print(df.shape)
print('Cantidad de registros:', df.shape[0])
print('Cantidad de variables:', df.shape[1])

### Variables o característas del dataset

In [ ]:
print(data.feature_names)

### Información del dataset

In [ ]:
df.info()

### Estadísticas del dataset

In [ ]:
df.describe(include='all')

### Función de evaluación (fitness): precisión media con validación cruzada

Esta función es el corazón del problema de optimización: las metaheurísticas (como recocido simulado o algoritmo genético) generan soluciones candidatas (máscaras binarias) y esta función les dice qué tan buena es cada una, basándose en la precisión de clasificación.



In [ ]:
def evaluate_solution(mask):
    # Esta función evalúa qué tan buena es una solución (máscara de selección de variables)
    # La entrada 'mask' es un vector binario del mismo tamaño que el número de características (features).
    # Por ejemplo, si mask = [1, 0, 1, 0], significa que solo se están usando las variables 0 y 2.

    if np.sum(mask) == 0:
        # Si no se selecciona ninguna variable (todos los valores en mask son 0),
        # no se puede entrenar un modelo. En ese caso, se devuelve precisión = 0.
        return 0

    # Selecciona solo las columnas de X donde mask == 1 (es decir, las variables activadas)
    X_selected = X[:, mask == 1]

    # Se elige un modelo de árbol de decisión como clasificador base
    clf = DecisionTreeClassifier()

    # Se evalúa la precisión promedio usando validación cruzada de 5 pliegues (5-fold cross-validation)
    # Esto ayuda a estimar qué tan bien generaliza la selección de variables sin sobreajuste
    score = cross_val_score(clf, X_selected, y, cv=5).mean()

    # Se retorna la precisión media como la medida de calidad (fitness) de la solución
    return score

### Recocido Simulado (Simulated Annealing)

Este algoritmo imita el proceso físico de enfriamiento de metales, permitiendo al sistema "explorar" soluciones peores al principio (para evitar quedarse atrapado en mínimos locales), y luego refinar progresivamente la búsqueda hasta encontrar una solución óptima o cercana.



In [ ]:
# Biblioteca estándar usada para selección aleatoria de elementos
import random

# Recocido simulado para selección de variables
def simulated_annealing(n_iterations=1000, initial_temp=1.0, cooling_rate=0.995):
    """
    Implementa el algoritmo de Recocido Simulado (Simulated Annealing) para seleccionar
    un subconjunto de variables que maximice la precisión de un clasificador.

    Parámetros:
    - n_iterations: número total de iteraciones a ejecutar.
    - initial_temp: temperatura inicial del sistema (controla la probabilidad de aceptar soluciones peores al inicio).
    - cooling_rate: factor por el cual se reduce la temperatura en cada iteración (debe estar entre 0 y 1).

    Retorna:
    - best: la mejor máscara binaria encontrada (variables seleccionadas).
    - best_score: precisión promedio (fitness) de esa mejor solución.
    """

    # Generar solución inicial aleatoria: vector binario (0 o 1) del mismo tamaño que el número de características
    current = np.random.randint(0, 2, size=n_features)

    # Evaluar la calidad (precisión) de la solución inicial
    current_score = evaluate_solution(current)

    # Guardar la mejor solución conocida hasta el momento (inicialmente es la misma)
    best = current.copy()
    best_score = current_score

    # Establecer temperatura inicial
    temp = initial_temp

    # Ciclo principal del algoritmo (iteraciones)
    for i in range(n_iterations):

        # Crear una solución vecina modificando una sola variable aleatoriamente
        neighbor = current.copy()
        idx = np.random.randint(n_features)  # escoger una posición aleatoria
        neighbor[idx] = 1 - neighbor[idx]    # cambiar de 0→1 o de 1→0

        # Evaluar la nueva solución vecina
        neighbor_score = evaluate_solution(neighbor)

        # Calcular la diferencia en rendimiento (delta)
        delta = neighbor_score - current_score

        # Criterio de aceptación tipo Metropolis:
        # - Si la nueva solución es mejor (delta > 0), se acepta siempre.
        # - Si es peor, se acepta con cierta probabilidad que depende de delta y la temperatura.
        if delta > 0 or np.exp(delta / temp) > np.random.rand():
            current = neighbor
            current_score = neighbor_score

            # Si además esta nueva solución es la mejor de todas, la guardamos
            if current_score > best_score:
                best = current.copy()
                best_score = current_score

        # Reducir la temperatura gradualmente (enfriamiento simulado)
        temp *= cooling_rate

        # Mostrar información de progreso cada 50 iteraciones y en la última
        if i % 50 == 0 or i == n_iterations - 1:
            print(f"Iteración {i}: Precisión = {current_score:.4f}, Temp = {temp:.4f}")

    # Al finalizar, se retorna la mejor solución encontrada
    return best, best_score

### Algoritmo Genético (Genetic Algorithm)

¿Qué hace este algoritmo?
* Crea una población inicial de soluciones (máscaras binarias de selección de variables).
* Evalúa su desempeño mediante validación cruzada.
* Selecciona los mejores individuos para formar la siguiente generación.
* Genera nuevos individuos aplicando cruce y mutación.
* Itera por varias generaciones, refinando las soluciones.
* Devuelve el mejor conjunto de variables al final del proceso.



In [ ]:
def genetic_algorithm(pop_size=20, generations=50, crossover_rate=0.8, mutation_rate=0.1):
    """
    Algoritmo genético para seleccionar un subconjunto óptimo de variables que maximice
    el rendimiento de un clasificador (en este caso, un árbol de decisión).

    Parámetros:
    - pop_size: número de individuos en la población.
    - generations: número total de generaciones que se van a evolucionar.
    - crossover_rate: probabilidad de realizar cruce entre padres.
    - mutation_rate: probabilidad de mutar un hijo.

    Retorna:
    - best_ind: el mejor individuo (máscara de variables seleccionadas).
    - score: precisión (fitness) de ese individuo.
    """

    # Paso 1: Inicializar la población con soluciones aleatorias (vectores binarios)
    population = [np.random.randint(0, 2, size=n_features) for _ in range(pop_size)]

    # Paso 2: Iterar por cada generación (ciclo evolutivo)
    for gen in range(generations):

        # Evaluar el desempeño (aptitud) de cada individuo de la población
        scores = [evaluate_solution(ind) for ind in population]

        # Selección: se escogen los mejores individuos para reproducirse (ordenados por puntaje)
        # np.argsort(scores)[-pop_size:] obtiene los índices de los mejores individuos
        selected = [population[i] for i in np.argsort(scores)[-pop_size:]]

        # Inicializar nueva población
        new_population = []

        # Paso 3: Generar nuevos individuos (hijos) hasta llenar la nueva población
        while len(new_population) < pop_size:
            # Selección aleatoria de dos padres entre los mejores
            parents = random.sample(selected, 2)

            # Cruce (recombinación genética) con cierta probabilidad
            if np.random.rand() < crossover_rate:
                # Seleccionar un punto de corte aleatorio
                point = np.random.randint(1, n_features - 1)
                # Crear dos hijos combinando partes de los padres
                child1 = np.concatenate((parents[0][:point], parents[1][point:]))
                child2 = np.concatenate((parents[1][:point], parents[0][point:]))
            else:
                # Si no hay cruce, los hijos son copias directas de los padres
                child1, child2 = parents

            # Mutación: cada hijo puede tener una variable invertida aleatoriamente
            for child in [child1, child2]:
                if np.random.rand() < mutation_rate:
                    idx = np.random.randint(n_features)
                    child[idx] = 1 - child[idx]  # invierte el bit (0 → 1 o 1 → 0)

                # Agregar el hijo a la nueva población
                new_population.append(child)

        # Reemplazar la población vieja por la nueva
        population = new_population[:pop_size]  # asegurar que no exceda el tamaño

        # Reporte de progreso: imprimir mejor puntuación de esta generación
        best_idx = np.argmax([evaluate_solution(ind) for ind in population])
        print(f"Generación {gen + 1}: Mejor precisión = {evaluate_solution(population[best_idx]):.4f}")

    # Paso 4: Después de todas las generaciones, retornar el mejor individuo encontrado
    best_ind = max(population, key=evaluate_solution)
    return best_ind, evaluate_solution(best_ind)

### Comparación final

In [ ]:
# Ejecutar recocido simulado
print("== Recocido simulado ==")
best_sa, score_sa = simulated_annealing()
print(f"Mejor precisión SA: {score_sa:.4f}")
print(f"Variables seleccionadas: {np.sum(best_sa)}")

# Mostrar nombres de las variables seleccionadas
selected_features_sa = data.feature_names[best_sa == 1]
print("Características seleccionadas (SA):")
for feature in selected_features_sa:
    print(f"- {feature}")

# ========================================================

# Ejecutar algoritmo genético
print("\n== Algoritmo genético ==")
best_ga, score_ga = genetic_algorithm()
print(f"Mejor precisión GA: {score_ga:.4f}")
print(f"Variables seleccionadas: {np.sum(best_ga)}")

# Mostrar nombres de las variables seleccionadas
selected_features_ga = data.feature_names[best_ga == 1]
print("Características seleccionadas (GA):")
for feature in selected_features_sa:
    print(f"- {feature}")